In [10]:
import os
import csv
import timeit
import tifffile
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET

# TODO: make a log file of all the files that were processed and any that failed

parent_folder_path = '/Volumes/T7/200DCE_240228_3xGFP-Ect2PH(PBC)_SFC/scope_folders'

def get_pixel_size(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    microns_per_pixel_element = root.find(".//PVStateValue[@key='micronsPerPixel']")
    microns_per_pixel = {}

    for indexed_value in microns_per_pixel_element.findall("./IndexedValue"):
        axis = indexed_value.attrib["index"]
        value = float(indexed_value.attrib["value"])
        microns_per_pixel[axis] = value 

    return microns_per_pixel['XAxis'], microns_per_pixel['YAxis']

def get_frame_rate(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    frames = root.findall('.//Frame')
    absolute_times = {}

    for index, frame in enumerate(frames, start=1):
        absolute_time = frame.attrib.get('absoluteTime')
        files = frame.findall('File')
        for file in files:
            filename = file.attrib.get('filename')
            if "000001.ome.tif" in filename:
                name = int(filename.split("_")[-3].split("e")[-1])
                absolute_times[name] = absolute_time

    num_frames = len(absolute_times)
        
    total_time = float(absolute_times[num_frames])
    framerate = total_time / num_frames

    return framerate

def make_log(
    directory: str, 
    logParams: dict
):
    '''
    Convert dictionary of parameters to a log file and save it in the directory
    '''
    logPath = os.path.join(directory, f"!image_conversion_log.txt")
    logFile = open(logPath, "w")                                    
    logFile.write("\n" + now.strftime("%Y-%m-%d %H:%M") + "\n")     
    for key, value in logParams.items():                            
        logFile.write('%s: %s\n' % (key, value))                    
    logFile.close()   

def create_hyperstack(folder_path):
    # convert and save the tiff stack
    all_files = []
    mip_folder_path = os.path.join(folder_path, "MIP")
    if os.path.exists(mip_folder_path) and os.path.isdir(mip_folder_path):
        files_in_mip = [os.path.join(mip_folder_path, file) for file in os.listdir(mip_folder_path)]
        all_files.extend(files_in_mip)

    channel_files = {}
    for file in all_files:
        channel_name = os.path.basename(file).split('_')[-2] 
        if channel_name not in channel_files:
            channel_files[channel_name] = []
        channel_files[channel_name].append(file)

    # Read images for each channel
    channel_images = {}
    for channel_name, files in channel_files.items():
        channel_images[channel_name] = [tifffile.imread(file) for file in files]

    # Stack images for each channel
    stacked_images = {}
    for channel_name, images in channel_images.items():
        stacked_images[channel_name] = np.stack(images)

    # Stack images across channels
    merged_images = np.stack(list(stacked_images.values()), axis=1)

    return merged_images

# performance tracker
start = timeit.default_timer()

# get the folders
folders = [folder for folder in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, folder))]
folders = sorted(folders)

# create the output folder
output_path = os.path.join(parent_folder_path, "!processed_images")
os.makedirs(output_path, exist_ok=True)

# log file
log_params = {'Files Not Processed': [],
              'Files Processed': []}

with tqdm(total = len(folders)) as pbar:
    pbar.set_description('Files processed:')
    for folder in folders:
        print('******'*10)
        try:
            folder_path = os.path.join(parent_folder_path, folder)
            
            # set the image name
            image_name = os.path.join(output_path, f"MAX_{folder}_raw.tif")

            if os.path.exists(image_name):
                print(f"{folder} already exists!")
                log_params['Files Not Processed'].append(f'{folder}: Already exists!')
                pass
            else:
                # get the xml file
                xml_file = [file for file in os.listdir(folder_path) if os.path.splitext(file)[1] == ".xml"]
                xml_file = os.path.join(folder_path, xml_file[0])

                # get the pixel size
                X_microns_per_pixel, Y_microns_per_pixel =  get_pixel_size(xml_file)

                # get the frame rate
                framerate = get_frame_rate(xml_file)

                # create the hyperstack
                hyperstack = create_hyperstack(folder_path)
                
                # save the hyperstack
                
                tifffile.imsave(
                    image_name, 
                    hyperstack,
                    imagej=True,
                    resolution=(1/X_microns_per_pixel, 1/Y_microns_per_pixel),
                    metadata={'axes': 'TCYX',
                            'finterval': framerate,
                            'mode': 'composite'}
                    )

                log_params['Files Processed'].append(folder)
                print(f"Successfully processed {folder}!")
            
        except Exception as e:
            log_params['Files Not Processed'].append(f'{folder}: {e}')
            pass

        pbar.update(1)
    
end = timeit.default_timer()
log_params["Time Elapsed"] = f"{end - start:.2f} seconds"

make_log(parent_folder_path, log_params)

print("Done!")


Files processed::   0%|          | 0/17 [00:00<?, ?it/s]

************************************************************


Files processed::   6%|▌         | 1/17 [00:00<00:13,  1.20it/s]

Successfully processed 200DCE_DC191_c-001!
************************************************************


Files processed::  12%|█▏        | 2/17 [00:01<00:09,  1.55it/s]

...

Successfully processed 200DCE_p78_c-010!
************************************************************


Files processed::  88%|████████▊ | 15/17 [00:14<00:01,  1.08it/s]

Successfully processed 200DCE_p78_c-011!
************************************************************


Files processed:: 100%|██████████| 17/17 [00:15<00:00,  1.07it/s]

Successfully processed 200DCE_p78_c-013!
************************************************************
Done!


In [ ]:
import os
import numpy as np
import tifffile

parent_folder_path = '/Volumes/T7/201DCE_240229_3xGFP-Ect2PH(PBC)_tagged-Ect2_SFC'

# Find all folders within the parent folder
folders = [folder for folder in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, folder))]

# Get all files in each folder
for folder in folders:
    try:
        all_files = []
        folder_path = os.path.join(parent_folder_path, folder)
        mip_folder_path = os.path.join(folder_path, "MIP")
        if os.path.exists(mip_folder_path) and os.path.isdir(mip_folder_path):
            files_in_mip = [os.path.join(mip_folder_path, file) for file in os.listdir(mip_folder_path)]
            all_files.extend(files_in_mip)

        ch1_files = [file for file in all_files if "Ch1" in file]
        ch2_files = [file for file in all_files if "Ch2" in file]

        ch1_image = []
        for file in ch1_files:
            image = tifffile.imread(file)
            ch1_image.append(image)

        ch2_image = []
        for file in ch2_files:
            image = tifffile.imread(file)
            ch2_image.append(image)

        # Convert the list to a numpy array
        ch1_image = np.stack(ch1_image)
        ch2_image = np.stack(ch2_image)
        
        output_path = os.path.join(parent_folder_path, "!processed_images")
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        merged_image = np.stack([ch1_image, ch2_image], axis=1)

        # Save the multi-frame image
        tifffile.imsave(os.path.join(output_path, f"MAX_{folder}_raw.tif"), merged_image)

        print(f"Successfully processed {folder}!")
    except Exception as e:
        print(f"Failed to process {folder}!")
        print(e)
        pass
    
print("Done!")     

In [66]:
import os
import xml.etree.ElementTree as ET


parent_folder_path = '/Users/domchom/random_movies/anilyzer_test_data/SFC_t-series'

# Find all folders within the parent folder
folders = [folder for folder in os.listdir(parent_folder_path) if os.path.isdir(os.path.join(parent_folder_path, folder))]

# Get all files in each folder
for folder in folders:
    all_files = []
    folder_path = os.path.join(parent_folder_path, folder)
    
    xml_file = [file for file in os.listdir(folder_path) if file.endswith(".xml")]
    xml_file = os.path.join(folder_path, xml_file[0])

    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    microns_per_pixel_element = root.find(".//PVStateValue[@key='micronsPerPixel']")

    # Extract microns per pixel values
    microns_per_pixel = {}
    for indexed_value in microns_per_pixel_element.findall("./IndexedValue"):
        axis = indexed_value.attrib["index"]
        value = float(indexed_value.attrib["value"])
        microns_per_pixel[axis] = value 

    print(microns_per_pixel)

    # Find all Frame elements
    frames = root.findall('.//Frame')

    absolute_times = {}

    # Extract and print the absolute time attribute value for each Frame
    for frame in frames:
        absolute_time = frame.attrib.get('absoluteTime')
        files = frame.findall('File')
        for file in files:
            filename = file.attrib.get('filename')
            if "000005.ome.tif" in filename:
                name = filename.split("_")[-3]
                name = name.split("e")[-1]
                name = int(name)
                absolute_times[name] = absolute_time

    num_frames = len(absolute_times)
    total_time = float(absolute_times[num_frames])
    
    framerate = total_time / num_frames

# Define the CSV file path
csv_file_path = '/path/to/save/csv/file.csv'

# Define the header of the CSV file
header = ['Folder Name', 'Pixel Size', 'Frame Rate']

# Create a list to store the data for each folder
data = []

# Iterate over the folders
for folder in folders:
    # Get the pixel size and frame rate for the folder
    pixel_size = microns_per_pixel['ZAxis']
    frame_rate = framerate
    
    # Append the folder name, pixel size, and frame rate to the data list
    data.append([folder, pixel_size, frame_rate])

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(data)

print("CSV file saved successfully!")
    
            
    
    
print("Done!")     

{'XAxis': 0.26614481409002, 'YAxis': 0.26614481409002, 'ZAxis': 1.0}
{'XAxis': 0.26614481409002, 'YAxis': 0.26614481409002, 'ZAxis': 1.0}
{'XAxis': 0.26614481409002, 'YAxis': 0.26614481409002, 'ZAxis': 1.0}
{'XAxis': 0.26614481409002, 'YAxis': 0.26614481409002, 'ZAxis': 1.0}


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/save/csv/file.csv'